In [2]:
import xarray as xr
from pathlib import Path

In [3]:
data_2d_z2 =  '/g/data/qx55/germany_node/d3hp003.zarr/P1D_mean_z10_atm.zarr'
ds_2 = xr.open_zarr(data_2d_z2)
ds_2

<xarray.Dataset> Size: 6TB
Dimensions:        (time: 425, cell: 12582912, crs: 1, pressure: 30,
                    soil_level: 5, pressure_rva: 3)
Coordinates:
  * crs            (crs) float32 4B nan
  * pressure       (pressure) int64 240B 5 10 20 50 ... 92500 95000 97500 100000
  * pressure_rva   (pressure_rva) int64 24B 16 18 23
  * soil_level     (soil_level) int64 40B 0 0 0 2 6
  * time           (time) datetime64[ns] 3kB 2020-01-02 ... 2021-03-01
Dimensions without coordinates: cell
Data variables: (12/58)
    clivi          (time, cell) float32 21GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    clt            (time, cell) float32 21GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    clwvi          (time, cell) float32 21GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    egpvi          (time, cell) float32 21GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    einvi          (time, cell) float32 21GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    ekhvi          (time, cell) float32 21GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    ...             ...
    ua             (time, pressure, cell) float32 642GB dask.array<chunksize=(4, 1, 262144), meta=np.ndarray>
    uas            (time, cell) float32 21GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    va             (time, pressure, cell) float32 642GB dask.array<chunksize=(4, 1, 262144), meta=np.ndarray>
    vas            (time, cell) float32 21GB dask.array<chunksize=(4, 262144), meta=np.ndarray>
    wa             (time, pressure, cell) float32 642GB dask.array<chunksize=(4, 1, 262144), meta=np.ndarray>
    zg             (time, pressure, cell) float32 642GB dask.array<chunksize=(4, 1, 262144), meta=np.ndarray>
Attributes:
    hiopy::parent:  P1D_mean_z11_atm

In [7]:
pr = ds_2['pr']  # (time, cell)
pr_day = pr.isel(time=2) * 86400 

In [8]:
sm = ds_2['mrso']  # (time, cell)
# pr_day = pr.isel(time=2) * 86400 

In [12]:
sm_sum = sm.sum(dim='soil_level')

In [13]:
# Compute 95th percentile along the 'time' dimension
pr_95 = pr.quantile(0.95, dim='time')

# Create a mask of values greater than the 95th percentile
pr_gt_95 = pr > pr_95


In [14]:
# Compute 95th percentile along the 'time' dimension
sm_95 = sm.quantile(0.95, dim='time')

# Create a mask of values greater than the 95th percentile
sm_gt_95 = sm > sm_95


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Step 1: Compute 95th percentiles along 'time' axis
sm_95 = sm.quantile(0.95, dim='time')
pr_95 = pr.quantile(0.95, dim='time')

# Step 2: Filter values greater than 95th percentile
sm_masked = sm.where(sm > sm_95)
pr_masked = pr.where(pr > pr_95)

# Step 3: Flatten and remove NaNs
sm_flat = sm_masked.values.flatten()
pr_flat = pr_masked.values.flatten()

# Mask NaNs to align pairs
import numpy as np
valid_mask = np.isfinite(sm_flat) & np.isfinite(pr_flat)

sm_valid = sm_flat[valid_mask]
pr_valid = pr_flat[valid_mask]

# Step 4: Scatter plot
plt.figure(figsize=(8,6))
plt.scatter(sm_valid, pr_valid, s=1, alpha=0.3, color='blue')
plt.xlabel("Soil Moisture (sm)", fontsize=12)
plt.ylabel("Precipitation (pr)", fontsize=12)
plt.title("Scatter plot: Values > 95th Percentile", fontsize=14)
plt.grid(True)
plt.tight_layout()
plt.show()
